# Euclidean Alignment + Weighted TL

## Load dataset

In [1]:
import sys

In [2]:
sys.path.append('D:\Google_Drive\JupyterNotebookProjects\bci-research\plan_c\csp')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import moabb.datasets
import moabb.paradigms


from copy import deepcopy
from csp.utils import subject_counter
from csp.preprocess import fir_bandpass, apply_bandpass, fetch_left_right_EEG
from csp.preprocess import split_EEG_one_class, process_s_data
from csp.feat_extraction import compute_Z, feat_vector, true_label
from csp.csp import CSP

In [4]:
dframe = pd.DataFrame

In [25]:
# First import the data

# Number of subject, n + 1 for iteration purpose (there are 9 subjects)
ns = 10

# Creating dict to store original data and modified data
# ori_data will serve as initial loaded data that will remain unchanged
ori_data = dict()
mod_data = dict() 

## Load Dataset

In [26]:
# Iter over all data path then store them in sub0X variable
for subj in range(1, 10):
    data_path = '../../datasets/BCICIV2a/A{:02d}T.npz'.format(subj)
    
    # Load EEG data from datapath and store into subj0X variabel then store into ori_dict
    # Then also fetch 's' (EEG data) into mod_data
    ori_data[subj] = np.load(data_path)
    mod_data[subj] = {}
    mod_data[subj]['s'] = deepcopy(ori_data[subj]['s'])
    mod_data[subj]['etyp'] = deepcopy(ori_data[subj]['etyp'])
    mod_data[subj]['epos'] = deepcopy(ori_data[subj]['epos'])
    
    # Remove last three EOG electrodes
    mod_data[subj]['s'] = np.delete(mod_data[subj]['s'], np.s_[22:], 1)

## Preprocessing

### Apply Bandpass Filter

In [27]:
from scipy.signal import firwin, freqs, lfilter

In [28]:
# Frequency sampling at 250 Hz
fs = 250

# Applying bandpass filter 8-30 Hz to all subjects raw EEG
b = fir_bandpass(51, low=8, high=30, fs=fs)

In [29]:
## PREPROCESSING ##

# Frequency sampling at 250 Hz
fs = 250

# Applying bandpass filter 8-30 Hz to all subjects raw EEG
b = fir_bandpass(51, low=8, high=30, fs=fs)

for subj in mod_data.keys():
    temp_raw_EEG = mod_data[subj]['s']
    
    mod_data[subj]['EEG_filtered'] = apply_bandpass(temp_raw_EEG, b)

In [30]:
dframe(mod_data[1]['EEG_filtered']).head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.020422,-0.022875,-0.020752,-0.024290,-0.024856,-0.026978,-0.014055,-0.021743,-0.024573,-0.027733,...,-0.025186,-0.022356,-0.023960,-0.029572,-0.028629,-0.025752,-0.020234,-0.024573,-0.024054,-0.019007
1,-0.028404,-0.031205,-0.030831,-0.033074,-0.033981,-0.033931,-0.020491,-0.029673,-0.036051,-0.038065,...,-0.031740,-0.030597,-0.034277,-0.040461,-0.039797,-0.034346,-0.029374,-0.034872,-0.033981,-0.029271
2,-0.022534,-0.027174,-0.026464,-0.029447,-0.028894,-0.029548,-0.017482,-0.027437,-0.032539,-0.034971,...,-0.024285,-0.029289,-0.032084,-0.038319,-0.037874,-0.032493,-0.028467,-0.033443,-0.032392,-0.028682
3,-0.027857,-0.027150,-0.032594,-0.035113,-0.038213,-0.037143,-0.019260,-0.029577,-0.040505,-0.041637,...,-0.035856,-0.031609,-0.039735,-0.045916,-0.046540,-0.041085,-0.032311,-0.039749,-0.039492,-0.030589
4,-0.064138,-0.066045,-0.067683,-0.079740,-0.082914,-0.087771,-0.040368,-0.065460,-0.080927,-0.091818,...,-0.087361,-0.067454,-0.079225,-0.097632,-0.098159,-0.089933,-0.064292,-0.082052,-0.082849,-0.061782


### Convert samples x n_electrodes -> no_trials x n_electrodes x samples 
Remember to process the filtered EEG with shape of samples x electrodes

In [44]:
# Iterate overall all_data
for subj in mod_data.keys():
    print('Processing for subject', subj)
    mod_data[subj]['all_trials'], mod_data[subj]['y'], _ = process_s_data(data=mod_data[subj], eeg_key='EEG_filtered', start_t=0.5, end_t=0.5, fs=250)

Processing for subject 1
Processing for subject 2
Processing for subject 3
Processing for subject 4
Processing for subject 5
Processing for subject 6
Processing for subject 7
Processing for subject 8
Processing for subject 9


In [46]:
mod_data[2]['all_trials'].shape

(144, 22, 0)

In [40]:
data = mod_data[1]
eeg_key = 'EEG_filtered'
start_t = 0.5
end_t = 3.5
fs=250

In [41]:
# Event type and position of subject
typ = data['etyp']
pos = data['epos']

# Grab position of each left (etype=769) and right (etype=770) class
pos_left = pos[typ==769]
pos_right = pos[typ==770]
all_pos = np.hstack([pos_left, pos_right])

# True label
y_left = np.zeros(len(pos_left))
y_right = np.ones(len(pos_right))
all_y = np.hstack([y_left, y_right])

# Sort them ascendingly based on event occurences
ids = np.argsort(all_pos)
all_pos = all_pos[ids]
all_y = all_y[ids]

fs=250

# Now convert 's' data into data of trials
s = data[eeg_key]

all_trials = []
for pos in all_pos:
    all_trials.append(s[(pos-1)+int(fs*start_t) : (pos-1)+int(fs*end_t)].T)

all_trials = np.array(all_trials) 

# # Return these
# return all_trials, all_y, all_pos

In [42]:
all_trials.shape

(144, 22, 750)